In [1]:
import dask
import json
import os
import random
import time
import pandas as pd
import dask.dataframe as dd
import dask.bag as db
from datetime import datetime
from dask import delayed
from dask.distributed import Client, progress, as_completed

C:\Users\Natalia\PycharmProjects\analiza_danych\.venv\Lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Zadanie 1

In [2]:
client = Client(n_workers=4)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 31.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59006,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 31.92 GiB
Comm: tcp://127.0.0.1:59028,Total threads: 3
Dashboard: http://127.0.0.1:59035/status,Memory: 7.98 GiB
Nanny: tcp://127.0.0.1:59009,


In [6]:
with open('Apache_2k.log', 'r') as f:
    lines = f.readlines()

def parse(inp: str):
    record = {}
    
    date_start = inp.find('[') + 1
    date_end = inp.find(']')
    date_s = slice(date_start, date_end)

    level_start = inp.find('[', date_end) + 1
    level_end = inp.find(']', level_start)
    level_s = slice(level_start, level_end)

    client_start = inp.find('[', level_end)
    client_end = inp.find(']', client_start)

    record["date"] = inp[date_s]    
    record["level"] = inp[level_s]
    record["client"] = "" if client_start == -1 else inp[client_start + 8: client_end]
    record["message"] = inp[client_end + 2:] if record["client"] else inp[level_end + 2:]
    
    return record

def convert_date(rec):
    rec["date"] = datetime.strptime(rec["date"], "%a %b %d %H:%M:%S %Y")

    return rec


output = []

for line in lines:
    record = parse(line)
    record = convert_date(record)
    output.append(list(record.values()))
    
df = pd.DataFrame(output, columns=["date", "level", "client", "message"])

df.head()

,date,level,client,message
0,2005-12-04 04:47:44,notice,,workerEnv.init() ok /etc/httpd/conf/workers2.p...
1,2005-12-04 04:47:44,error,,mod_jk child workerEnv in error state 6\n
2,2005-12-04 04:51:08,notice,,jk2_init() Found child 6725 in scoreboard slot...
3,2005-12-04 04:51:09,notice,,jk2_init() Found child 6726 in scoreboard slot...
4,2005-12-04 04:51:09,notice,,jk2_init() Found child 6728 in scoreboard slot...


In [ ]:
with open('Apache_2k.log', 'r') as f:
    lines = f.readlines()

def parse_date(inp: str):
    date_start = inp.find('[') + 1
    date_end = inp.find(']')
    return inp[date_start:date_end]

def parse_level(inp: str, date_end):
    level_start = inp.find('[', date_end) + 1
    level_end = inp.find(']', level_start)
    return inp[level_start:level_end]

def parse_client(inp: str, level_end):
    client_start = inp.find('[', level_end)
    if client_start == -1:
        return ""
    client_end = inp.find(']', client_start)
    return inp[client_start + 8: client_end]

def parse_message(inp: str, level_end, client):
    client_end = inp.find(']', level_end)
    if client:
        return inp[client_end + 2:]
    return inp[level_end + 2:]

def convert_date(date_str):
    return datetime.strptime(date_str, "%a %b %d %H:%M:%S %Y")

def parse_line(line):
    date = parse_date(line)
    level = parse_level(line, line.find(']'))
    client = parse_client(line, line.find(']', line.find(']') + 1))
    message = parse_message(line, line.find(']', line.find(']') + 1), client)
    
    return {"date": date, "level": level, "client": client, "message": message}

parsed_lines = [parse_line(line) for line in lines]

def process_record(record):
    record["date"] = convert_date(record["date"])
    return list(record.values())

processed_output = [process_record(record) for record in parsed_lines]
output = dask.compute(*processed_output)

df = pd.DataFrame(output, columns=["date", "level", "client", "message"])
df.head()

Zadanie 2

In [7]:
DATAPATH = './data'
os.makedirs(DATAPATH, exist_ok=True)

b = dask.datasets.make_people(npartitions=100, records_per_partition=10000)

def expired_card(record):
    exp_date = record.get('cc_exp', '00/00')
    month, year = map(int, exp_date.split('/'))
    return (year < 2024) or (year == 2024 and month < 10)

expired_records = b.filter(expired_card)

In [8]:
expired_records = expired_records.repartition(10)
expired_records.map(json.dumps).to_textfiles(os.path.join('data', 'expired_*.json'))

['C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_0.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_1.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_2.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_3.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_4.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_5.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_6.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_7.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_8.json',
 'C:/Users/Natalia/PycharmProjects/analiza_danych/data/expired_9.json']

Zadanie 3

In [9]:
adults = b.filter(lambda record: record['age'] >= 18).repartition(1)
df = adults.to_dataframe().compute()

In [10]:
def to_dataframe_record(record):
    df_record = {
        'first_name': record['name'][0],
        'last_name': record['name'][1],
        'age': record['age'],
        'occupation': record['occupation'],
        'telephone': record['telephone'],
        'address': record['address']['address'],
        'city': record['address']['city'],
        'credit_card_number': record['credit-card']['number'],
        'credit_card_expiration': record['credit-card']['expiration-date']
    }
    return df_record

In [11]:
df = adults.map(to_dataframe_record).to_dataframe()
df.to_parquet('data/adults')
df.head()

C:\Users\Natalia\PycharmProjects\analiza_danych\.venv\Lib\site-packages\dask\dataframe\io\parquet\core.py:1249: FutureWarning: Could not find pyarrow; falling back to fastparquet, which is deprecated and will be removed in a future release.
  warnings.warn(


,first_name,last_name,age,occupation,telephone,address,city,credit_card_number,credit_card_expiration
0,Francina,Thompson,60,Sound Artist,+1-906-599-8213,961 English Run,Mansfield,5549 7494 8327 6373,08/17
1,Sandie,Robbins,21,Floor Layer,+18471806511,549 Woolsey Loop,Oceanside,3418 081221 19666,06/23
2,Columbus,Santana,112,Sail Maker,+16318277624,442 Cuesta Station,Kingsville,4731 9637 7473 3197,08/22
3,Hae,Travis,57,Linguist,+12064371716,394 Russ Row,Tahlequah,3421 410484 88554,10/22
4,Jc,Leonard,56,Paediatrician,+16603951809,248 Storey Creek,Bloomington,4217 9032 0500 0363,01/24
